In [439]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Sigmoid Activation Function:
$$\sigma(x) = \frac{1} {1 + e^{-x}}$$

Derivative of Activation Function:
$$\sigma'(x) = \sigma'(x)(1-\sigma'(x))$$

In [440]:
class SimpleNeuralNetwork:
    
    # Simple 2 layer neural network
    # First layer has as a node for each feature 
    # Second layer has a configurable number of nodes that map to 1 output
    def __init__(self, num_of_features, second_layer_size):
        self.weights1 = np.random.rand(num_of_features, second_layer_size)
        self.weights2 = np.random.rand(second_layer_size, 1)
    
    def activation_func(self, x):
        return 1/(1+np.exp(-x))
    
    def activation_func_derivative(self, x):
        return self.activation_func(x)*(1-self.activation_func(x))
        
    def predict(self, features):
        results1 = self.activation_func(features.dot(self.weights1))
        results2 = self.activation_func(results1.dot(self.weights2))
        return results2;

Mean Squared Error:
$$MSE = \sum_{i=1}^{n}(y_i-\sigma(w(\sigma(wx+b))+b))^2$$

In [441]:
class Trainer:
    
    def get_errors(self, labels, predictions):
        return labels - predictions
    
    def get_mean_squared_error(self, errors):
        return np.sum(np.square(errors))/errors.size 
        
    def feedforward(self, nn, features):
        results1 = nn.activation_func(features.dot(nn.weights1))
        results2 = nn.activation_func(results1.dot(nn.weights2))
        return results1, results2
    
    def backpropagate(self, nn, features, results1, results2, errors, learning_rate):
        weights2_delta = (2/errors.size)*results1.T.dot(errors*nn.activation_func_derivative(results2))
        weights1_delta = (2/errors.size)*features.T.dot(((errors*nn.activation_func_derivative(results2)).dot(nn.weights2.T))*(nn.activation_func_derivative(results1)))
                
        nn.weights2 += learning_rate*weights2_delta
        nn.weights1 += learning_rate*weights1_delta
        
    #Batch gradient descent because all data is being used for a single step
    def train(self, nn, features, labels, learning_rate, epochs):
        for epoch in range(epochs):
            results1, results2 = self.feedforward(nn, features)
            errors = self.get_errors(labels, results2)
            mean_squared_errors = self.get_mean_squared_error(errors)
            print("At epoch:", epoch, ", MSE = ", mean_squared_errors) 
            self.backpropagate(nn, features, results1, results2, errors, learning_rate)

In [442]:
trainer = Trainer()

In [443]:
# Creating some simple training data just to check basic funcationality
simple_features = np.array([[-3,-3],
                            [-3,3],
                            [3,-3],
                            [3,3]])

simple_labels = np.array([[0], [1], [1], [1]])

In [444]:
simple_neural_network = SimpleNeuralNetwork(2, 10)
trainer.train(simple_neural_network, simple_features, simple_labels, 0.1, 100)

At epoch: 0 , MSE =  0.1047483994170382
At epoch: 1 , MSE =  0.10404185273285824
At epoch: 2 , MSE =  0.10334760784849266
At epoch: 3 , MSE =  0.10266553449432299
At epoch: 4 , MSE =  0.10199549741584486
At epoch: 5 , MSE =  0.10133735682169775
At epoch: 6 , MSE =  0.10069096881719221
At epoch: 7 , MSE =  0.10005618582265406
At epoch: 8 , MSE =  0.09943285697604078
At epoch: 9 , MSE =  0.0988208285194168
At epoch: 10 , MSE =  0.09821994416899248
At epoch: 11 , MSE =  0.09763004546854549
At epoch: 12 , MSE =  0.09705097212614189
At epoch: 13 , MSE =  0.09648256233417005
At epoch: 14 , MSE =  0.09592465307278153
At epoch: 15 , MSE =  0.09537708039690987
At epoch: 16 , MSE =  0.09483967970710394
At epoch: 17 , MSE =  0.09431228600447088
At epoch: 18 , MSE =  0.09379473413007426
At epoch: 19 , MSE =  0.09328685898917727
At epoch: 20 , MSE =  0.09278849576075647
At epoch: 21 , MSE =  0.09229948009274286
At epoch: 22 , MSE =  0.09181964828347022
At epoch: 23 , MSE =  0.09134883744983177
At e

In [445]:
# Creating some simple training data just to check basic funcationality
simple_test_data = np.array([[-10,-10],
                             [-10,10],
                             [10,-10],
                             [10,10]])

simple_test_labels = np.array([[0], [1], [1], [1]])
predicted_values = simple_neural_network.predict(some_test_data)
print("Predicted Values:", predicted_values)
errors = trainer.get_errors(simple_test_labels, predicted_values)
mean_squared_error = trainer.get_mean_squared_error(errors);
print("Mean Squared Error:", mean_squared_error)

Predicted Values: [[0.501117  ]
 [0.93085115]
 [0.89392546]
 [0.9912234 ]]
Mean Squared Error: 0.06680716232442688


In [446]:
# Okay, now that it seems to be working, lets test with some more complex data
# Using data from here: http://archive.ics.uci.edu/ml/datasets/Abalone
df = pd.read_csv("abalone.csv")
df.head()

,Type,LongestShell,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [447]:
# Doing a bit of preprocessing
def string_to_binary(string):
    if string == "F":
        return 1
    else:
        return 0
    
df = df.loc[df['Type'] != "I"]
df['Type'] = df['Type'].apply(string_to_binary)
df.head()

,Type,LongestShell,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
6,1,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,20


In [448]:
# Splitting into training and testing data
training_data, testing_data = train_test_split(df, test_size=0.2)
training_features = training_data.iloc[:, ~training_data.columns.isin(['Type'])].values
training_labels = training_data['Type'].values.reshape(len(training_features),1)
testing_features = testing_data.iloc[:, ~testing_data.columns.isin(['Type'])].values
testing_labels = testing_data['Type'].values.reshape(len(testing_features),1)

In [449]:
# Training
classification_neural_network = SimpleNeuralNetwork(8, 10)
trainer.train(classification_neural_network, training_features, training_labels, 0.1, 1000)

At epoch: 0 , MSE =  0.5266108293806938
At epoch: 1 , MSE =  0.5234228396514641
At epoch: 2 , MSE =  0.5194999169322192
At epoch: 3 , MSE =  0.514692861835195
At epoch: 4 , MSE =  0.5088408359546802
At epoch: 5 , MSE =  0.5017848546741025
At epoch: 6 , MSE =  0.49338840536151746
At epoch: 7 , MSE =  0.48356120921741885
At epoch: 8 , MSE =  0.4722798789843779
At epoch: 9 , MSE =  0.4596013606589475
At epoch: 10 , MSE =  0.4456683122611978
At epoch: 11 , MSE =  0.4307064480229463
At epoch: 12 , MSE =  0.4150133115005958
At epoch: 13 , MSE =  0.398938307649091
At epoch: 14 , MSE =  0.38285556496196305
At epoch: 15 , MSE =  0.3671332019372483
At epoch: 16 , MSE =  0.35210373972231174
At epoch: 17 , MSE =  0.3380402654532311
At epoch: 18 , MSE =  0.32514159606315524
At epoch: 19 , MSE =  0.3135276074936778
At epoch: 20 , MSE =  0.30324377394273444
At epoch: 21 , MSE =  0.29427243098689543
At epoch: 22 , MSE =  0.2865476502873667
At epoch: 23 , MSE =  0.27997083436300385
At epoch: 24 , MSE =

At epoch: 200 , MSE =  0.24894583093579442
At epoch: 201 , MSE =  0.24894843864752794
At epoch: 202 , MSE =  0.24895104440273336
At epoch: 203 , MSE =  0.24895364817195362
At epoch: 204 , MSE =  0.24895624992546922
At epoch: 205 , MSE =  0.24895884963329634
At epoch: 206 , MSE =  0.24896144726518551
At epoch: 207 , MSE =  0.24896404279061982
At epoch: 208 , MSE =  0.24896663617881368
At epoch: 209 , MSE =  0.24896922739871127
At epoch: 210 , MSE =  0.24897181641898458
At epoch: 211 , MSE =  0.2489744032080328
At epoch: 212 , MSE =  0.2489769877339801
At epoch: 213 , MSE =  0.24897956996467446
At epoch: 214 , MSE =  0.24898214986768658
At epoch: 215 , MSE =  0.24898472741030808
At epoch: 216 , MSE =  0.2489873025595503
At epoch: 217 , MSE =  0.24898987528214292
At epoch: 218 , MSE =  0.24899244554453293
At epoch: 219 , MSE =  0.24899501331288287
At epoch: 220 , MSE =  0.24899757855306995
At epoch: 221 , MSE =  0.24900014123068465
At epoch: 222 , MSE =  0.24900270131102956
At epoch: 223 

At epoch: 410 , MSE =  0.24938348219136233
At epoch: 411 , MSE =  0.24938462232853698
At epoch: 412 , MSE =  0.24938574919805817
At epoch: 413 , MSE =  0.24938686274684171
At epoch: 414 , MSE =  0.24938796292230073
At epoch: 415 , MSE =  0.24938904967235598
At epoch: 416 , MSE =  0.24939012294544488
At epoch: 417 , MSE =  0.24939118269053173
At epoch: 418 , MSE =  0.24939222885711712
At epoch: 419 , MSE =  0.24939326139524756
At epoch: 420 , MSE =  0.2493942802555255
At epoch: 421 , MSE =  0.2493952853891187
At epoch: 422 , MSE =  0.24939627674777018
At epoch: 423 , MSE =  0.24939725428380727
At epoch: 424 , MSE =  0.24939821795015218
At epoch: 425 , MSE =  0.2493991677003307
At epoch: 426 , MSE =  0.24940010348848238
At epoch: 427 , MSE =  0.24940102526936972
At epoch: 428 , MSE =  0.2494019329983882
At epoch: 429 , MSE =  0.24940282663157515
At epoch: 430 , MSE =  0.24940370612561974
At epoch: 431 , MSE =  0.24940457143787187
At epoch: 432 , MSE =  0.2494054225263525
At epoch: 433 , 

At epoch: 643 , MSE =  0.24927597215854208
At epoch: 644 , MSE =  0.2492742810450031
At epoch: 645 , MSE =  0.2492725855499748
At epoch: 646 , MSE =  0.24927088576060394
At epoch: 647 , MSE =  0.24926918176367532
At epoch: 648 , MSE =  0.24926747364560392
At epoch: 649 , MSE =  0.24926576149242777
At epoch: 650 , MSE =  0.24926404538980032
At epoch: 651 , MSE =  0.24926232542298374
At epoch: 652 , MSE =  0.24926060167684178
At epoch: 653 , MSE =  0.2492588742358331
At epoch: 654 , MSE =  0.24925714318400466
At epoch: 655 , MSE =  0.24925540860498524
At epoch: 656 , MSE =  0.24925367058197906
At epoch: 657 , MSE =  0.24925192919775987
At epoch: 658 , MSE =  0.24925018453466466
At epoch: 659 , MSE =  0.24924843667458815
At epoch: 660 , MSE =  0.24924668569897696
At epoch: 661 , MSE =  0.24924493168882397
At epoch: 662 , MSE =  0.249243174724663
At epoch: 663 , MSE =  0.24924141488656398
At epoch: 664 , MSE =  0.24923965225412725
At epoch: 665 , MSE =  0.24923788690647905
At epoch: 666 , 

At epoch: 871 , MSE =  0.24889307218647705
At epoch: 872 , MSE =  0.24889168693686933
At epoch: 873 , MSE =  0.24889030506318457
At epoch: 874 , MSE =  0.2488889265611688
At epoch: 875 , MSE =  0.24888755142643684
At epoch: 876 , MSE =  0.24888617965447413
At epoch: 877 , MSE =  0.24888481124063916
At epoch: 878 , MSE =  0.24888344618016536
At epoch: 879 , MSE =  0.24888208446816315
At epoch: 880 , MSE =  0.24888072609962253
At epoch: 881 , MSE =  0.2488793710694142
At epoch: 882 , MSE =  0.24887801937229287
At epoch: 883 , MSE =  0.24887667100289798
At epoch: 884 , MSE =  0.2488753259557568
At epoch: 885 , MSE =  0.24887398422528548
At epoch: 886 , MSE =  0.24887264580579196
At epoch: 887 , MSE =  0.2488713106914772
At epoch: 888 , MSE =  0.24886997887643725
At epoch: 889 , MSE =  0.24886865035466568
At epoch: 890 , MSE =  0.24886732512005474
At epoch: 891 , MSE =  0.2488660031663979
At epoch: 892 , MSE =  0.24886468448739113
At epoch: 893 , MSE =  0.24886336907663528
At epoch: 894 , 

In [450]:
# Checking performance on testing data
predicted_values = classification_neural_network.predict(testing_features)
errors = trainer.get_errors(testing_labels, predicted_values)
print("Mean Squared Error:", trainer.get_mean_squared_error(errors))

Mean Squared Error: 0.24939412897525498
